In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
df2 = pd.read_csv('../StarFish/data/streamers_clean.csv')

In [11]:
df.shape

(2450, 8)

In [12]:
df2.shape

(2450, 8)

In [17]:
test = pd.read_csv('top_2500_2')

In [18]:
test

,User,new_date,new_time,Duration,AVG Viewers,MAX Viewers,Followers Gain,Live Views,Title,daytime_classifier
0,Gotaga,2016-11-17,13:46:53,268,810,1321,0,0,NaN,2
1,Gotaga,2016-11-17,21:28:22,416,838,1292,134,8579,[FR] GOTAGA l Objectif le TOP 1 ! w/ Carbon / ...,3
2,Gotaga,2016-11-18,23:44:36,290,726,1059,86,3509,[FR] GOTAGA l BR1 GUNFIGHT LOOSEFIGHT,3
3,Gotaga,2016-11-20,02:06:35,158,551,781,100,3649,[FR] GOTAGA l BR1 GUNFIGHT LOOSEFIGHT,0
4,Gotaga,2016-11-20,23:59:33,385,802,1261,76,2447,[FR] GOTAGA l Shoot Pas Radical Oui Oui,3
...,...,...,...,...,...,...,...,...,...,...
100179,Pimpeano,2021-05-21,01:19:46,385,5153,8780,2070,60228,Chillstream ☕ 528/550 subs ☕ se charla un rato pa,0
100180,Pimpeano,2021-05-22,22:04:36,190,24652,31751,2048,85080,TRYHARDEAS O PILETAZO #1 🏆 Premios: 100 subs +...,3
100181,Pimpeano,2021-05-30,03:22:44,142,7195,10321,1876,43344,ON HASTA EL MILLON 🐱 !yt,0
100182,Pimpeano,2021-05-31,22:52:53,152,6682,9542,861,30295,MAÑANA ARRANCAMOS EN BOOYAH 🍀 Hoy timba !bitsl...,3


# trying to scrape historical data

## Scrape data # ez

In [1]:
def time_assignment(x):
    '''Support function to classify streams by time when they are on air'''
    midnight = 24
    morning = 6 
    noon = 12 
    evening = 18
    if morning <= x.hour < noon:
        return 'morning'
    elif noon <= x.hour < evening:
        return 'afternoon'
    elif evening <= x.hour < midnight:
        return 'evening'
    else:
        return 'night'

In [2]:
def parse_streams(usernames):
    '''For all usernames input as a list, retrieve their stats for all streams and classify by daytime'''
    import time
    t=1
    count = 0
    df_all = pd.DataFrame(columns=['Stream', 'Duration', 'AVG Viewers', 'MAX Viewers', 'Followers Gain', 'Live Views', 'Title', 'Games', 'Username'])
    for user in usernames:
        count += 1
        url = f"https://twitchtracker.com/{user}/streams"
        headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        df = pd.read_html(str(soup))[0]
        df['Username'] = f'{user}'
        temp_df = pd.concat([df_all, df])

        temp_df['Stream'] = pd.to_datetime(temp_df['Stream'])
        temp_df['Duration'] = temp_df['Duration'].apply(lambda x: int(x))
        temp_df['AVG Viewers'] = temp_df['AVG Viewers'].apply(lambda x: int(x))
        temp_df['MAX Viewers'] = temp_df['MAX Viewers'].apply(lambda x: int(x))
        temp_df['Followers Gain'] = temp_df['Followers Gain'].apply(lambda x: int(x))
        temp_df['Live Views'] = temp_df['Live Views'].apply(lambda x: int(x))
        new_dates, new_times = zip(*[(d.date(), d.time()) for d in temp_df['Stream']])
        df_clean = temp_df.assign(new_date=new_dates, new_time=new_times)
        df_clean = df_clean[['User', 'new_date', 'new_time', 'Duration', 'AVG Viewers', 'MAX Viewers', 'Followers Gain', 'Live Views', 'Title']]
        df_clean['daytime_classifier'] = df_clean['new_time'].apply(lambda x: time_assignment(x))
        if count % 10 == 0:
            print(count)
        time.sleep(t)
    return df_clean
# df.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/top100_streams', index=False)

In [9]:
usernames= df['username'].to_list()

In [10]:
streams_df = parse_streams(usernames)

KeyboardInterrupt: 

## putting all df together

In [23]:
df_top_2450 = pd.DataFrame(columns=['User', 'new_date', 'new_time', 'Duration', 'AVG Viewers', 'MAX Viewers', 'Followers Gain', 'Live Views', 'Title'])
for i in range(1, 16):
    df = pd.read_csv(f'top_2500_{i}')
    df_top_2450 = pd.concat([df_top_2450, df])

In [25]:
df_top_2450.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/top_2450', index=False)

## downsizing the df so it can be uploaded to git

In [26]:
df_top_2000 = df_top_2450[:2000]

In [28]:
df_top_2000.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/top_2000', index=False)

## Trying to match somehow the time of stream with the game on the twitchtracker page

In [19]:
# filter for only agurin user
agurin = temp_df[temp_df['User'] == 'agurin']

zeiten = agurin['Stream'].to_list()

zeiten

In [70]:
## not working out ...

url = f"https://twitchtracker.com/agurin/streams"
headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

soup.findAll(class_='games hidden-xs hidden-sm')

[img.text for td in soup.find_all('img title', class_='games hidden-xs hidden-sm')]

soup.find_all('img title', class_='games hidden-xs hidden-sm')

soup.findAll('2017-08-14')

soup.find_all('a').find('src').get('title')

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?